# Paper

In [1]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import time
from xgboost import XGBClassifier
from xgboost import XGBRegressor
import torch.nn as nn
import tqdm
import torch
import sys
sys.path.append('/work/users/d/d/ddinh/aaco/src')
from load_dataset import load_adni_data
from cvar_sensing.utils import prepare_time_series, batch_interp_nd
import torch

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
from tensorflow.keras.metrics import AUC

2024-12-22 21:53:49.450024: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-22 21:53:49.895829: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-22 21:53:50.063144: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-22 21:53:51.194940: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-22 21:53:55.587239: W tensorflow/compiler/tf2

In [2]:
X_train_digits = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_digits.npy')
X_train_counter = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_counter.npy')
y_train = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_labels.npy')
X_test_digits = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_digits.npy')
X_test_counter = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_counter.npy')
y_test = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_labels.npy')

train_y = np.eye(np.max(y_train) + 1)[y_train]
y_test = np.eye(np.max(y_test) + 1)[y_test]

train_x = np.concatenate([X_train_digits, X_train_counter], axis=1)
X_test = np.concatenate([X_test_digits, X_test_counter], axis=1)

In [3]:
classifier = XGBClassifier(n_estimators=256, device='gpu')
classifier.load_model('/work/users/d/d/ddinh/aaco/models/synthetic_raw.model')

In [4]:
# todo: implement progressively acquire features version 

In [ ]:
from sklearn.model_selection import train_test_split

class EmbeddingModels(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.num_features = 10 * 2
        self.base_model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(self.num_features,)),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
        ])
        self.embedding_head = tf.keras.layers.Dense(self.num_features + 1)
        self.prediction_head = tf.keras.layers.Dense(self.num_features + 1)

    def call(self, x):
        x = self.base_model(x)
        embedding_space = self.embedding_head(x)
        prediction = self.prediction_head(x)
        return embedding_space, prediction, x
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'num_features': self.num_features,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


@tf.function
def embedding_loss(distribution_true, distribution_embedding):
    return tf.keras.losses.CategoricalCrossentropy(from_logits=True)(distribution_true, distribution_embedding)

@tf.function
def prediction_loss(y_true, y_pred):
    return tf.keras.losses.CategoricalCrossentropy(from_logits=True)(y_true, y_pred)

def x_masked(x, d=20):
    masks = np.concatenate(
        [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(int(d/3))], 1, keepdims=True) for _ in range(x.shape[0])], 1
        )
    masks = np.float32(masks.T)

    masks_zero = np.sum(masks, axis=1) == 0
    
    while np.sum(masks_zero) > 0:
        # masks[masks_zero] = np.float32(np.concatenate(
        #     [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(np.sum(masks_zero))], 1
        #     )).T
        masks[masks_zero] = np.float32(np.concatenate(
            [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(int(d/3))], 1, keepdims=True) for _ in range(np.sum(masks_zero))], 1
            )).T
        
        masks_zero = np.sum(masks, axis=1) == 0
        
    return np.copy(x), masks


def compute_loss_timestep(y_true, x_data, mask, classifier, acquisition_cost, loss_function, num_timestamps=10, num_modalities=2):
    y_pred = classifier.predict_proba(np.concatenate([x_data * mask, mask], axis=1))
    y_pred = torch.Tensor(y_pred)
    if isinstance(y_true, tf.Tensor):
        y_true = torch.Tensor(y_true.numpy())
    else:
        y_true = torch.Tensor(y_true)
    total_cost = torch.zeros(mask.shape[0])
    total_cost += mask.sum(axis=1) * acquisition_cost
    total_loss = loss_function(y_pred, y_true).numpy() + total_cost.numpy()
    return total_loss

def get_potential_features(x, y, classifier, prev_masks, acquisition_cost, d=20, num_masks=1500, topk=5, num_timestamps=10, num_modalities=2):
    new_masks = np.concatenate(
        [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(num_masks)], 1
        )
    new_masks = np.float32(new_masks.T)

    # repeat for parallelization
    x_rep = np.repeat(x, num_masks, axis=0)
    y_rep = np.repeat(y, num_masks, axis=0)
    new_masks = np.concatenate([new_masks for _ in range(x.shape[0])], 0)
    prev_masks_rep = np.repeat(prev_masks, num_masks, axis=0)

    # combine previous masks with new masks
    N = x_rep.shape[0]
    segments_previous = prev_masks_rep.reshape(N, num_modalities, num_timestamps)
    segments_after = new_masks.reshape(N, num_modalities, num_timestamps)
    last_indices = np.where(segments_previous == 1, np.arange(num_timestamps), -1)
    last_indices_per_segment = np.max(last_indices, axis=2)  
    final_last_index = np.max(last_indices_per_segment, axis=1)  
    mask = np.arange(num_timestamps)[None, None, :] > final_last_index[:, None, None]
    final_segments = np.where(mask, segments_after, segments_previous)
    final = final_segments.reshape(N, num_timestamps * num_modalities)
    
    # compute loss for current and future masks
    current_loss = compute_loss_timestep(y, x, prev_masks, classifier, acquisition_cost, nn.CrossEntropyLoss(reduction='none'))
    future_loss = compute_loss_timestep(y_rep, x_rep, final, classifier, acquisition_cost, nn.CrossEntropyLoss(reduction='none'))
    
    # get the top k min loss
    top_k_sets = []
    distributions = []
    terminations = []
    for i in range(x.shape[0]):
        min_current_loss = current_loss[i]
        single_sample_future_loss = np.copy(future_loss[i * num_masks: (i + 1) * num_masks])
        single_sample_future_set = np.copy(final[i * num_masks: (i + 1) * num_masks])
        
        top_k_loss = np.argsort(single_sample_future_loss)
        min_k_future_loss = single_sample_future_loss[top_k_loss]
        min_k_subsets = single_sample_future_set[top_k_loss]
        
        unique_min_k_subsets, unique_indices = np.unique(min_k_subsets, axis=0, return_index=True)
        unique_min_k_future_loss = min_k_future_loss[unique_indices]
        
        sorted_unique_indices = np.argsort(unique_min_k_future_loss)
        unique_min_k_future_loss = unique_min_k_future_loss[sorted_unique_indices]
        unique_min_k_subsets = unique_min_k_subsets[sorted_unique_indices]
        
        if unique_min_k_subsets.shape[0] <= 1:
            topk_min = topk
            if np.equal(unique_min_k_subsets[0], prev_masks[i]).all():
                termination = np.ones(topk)
                unique_min_k_subsets = np.repeat(prev_masks[i][None, :], topk, axis=0)
                subset_losses = np.ones(topk) * min_current_loss
            else:
                if unique_min_k_future_loss[0] >= min_current_loss:
                    termination = np.ones(topk)
                    unique_min_k_subsets = np.repeat(prev_masks[i][None, :], topk, axis=0)
                    subset_losses = np.ones(topk) * min_current_loss
                else: 
                    termination = np.zeros(topk)
                    unique_min_k_subsets = np.repeat(unique_min_k_subsets[0], topk, axis=0)
                    subset_losses = np.repeat(unique_min_k_future_loss[0], topk)
        else: 
            topk_min = min(topk, unique_min_k_subsets.shape[0])
            termination = np.zeros(topk_min)
            subset_losses = unique_min_k_future_loss[:topk_min]

            for j in range(topk_min):
                if unique_min_k_future_loss[j] >= min_current_loss:
                    termination[j] = 1
                    unique_min_k_subsets[j] = prev_masks[i]
                    
        unique_min_k_subsets = unique_min_k_subsets[:topk_min]
        unique_min_k_subsets[:,:d] -= prev_masks[i]
        # get the first 1 in each segment
        # todo: consider changing this into a for loop, might be faster
        n = unique_min_k_subsets.shape[0]
        segment_size = num_timestamps
        num_segments = num_modalities
        segments = unique_min_k_subsets.reshape(n, num_segments, segment_size)
        first_one_indices = np.where(segments == 1, np.arange(segment_size), np.inf)
        first_one_per_segment = np.min(first_one_indices, axis=2)
        first_one_per_segment[first_one_per_segment == np.inf] = -1
        global_first_one = np.min(
            np.where(first_one_per_segment != -1, first_one_per_segment, np.inf), axis=1
        )
        global_first_one = np.where(global_first_one == np.inf, -1, global_first_one)
        keep_indices = (first_one_indices == global_first_one[:, None, None])
        transformed_masks = np.zeros_like(unique_min_k_subsets)
        transformed_masks[keep_indices.reshape(unique_min_k_subsets.shape)] = 1
        unique_min_k_subsets = np.copy(transformed_masks)
        unique_min_k_subsets = np.concatenate([unique_min_k_subsets, termination[:, None]], axis=1) # shape (topk, d+1)
        
        # turn into probabilities
        unique_min_k_subsets = unique_min_k_subsets / np.sum(unique_min_k_subsets, axis=1)[:, None]
        subset_losses = subset_losses[:topk_min]
        weights = np.exp(-subset_losses[:topk_min])
        weights /= np.sum(weights)
        distribution = np.sum(unique_min_k_subsets * weights[:, None], axis=0) 
        
        distributions.append(distribution)
        terminations.append(termination)

    distributions = np.array(distributions) # shape (batch_size, d)
    distributions /= (distributions.sum(axis=1, keepdims=True) + 1e-12) # normalize for numerical stability
    # print("check sum of distributions: ", np.sum(distributions, axis=1))
    
    next_action = np.array([np.random.choice(distributions.shape[1], p=distributions[i]) for i in range(distributions.shape[0])])

    terminations = np.concatenate(terminations)
    print("Percentage of terminated: ", np.mean(terminations))
    
    return top_k_sets, distributions, next_action
    
# @tf.function
def train_step(model, optimizer, x, y, classifier, acquisition_cost, alpha=1):
    x, prev_masks = x_masked(x)
    top_k_sets, distributions, next_feature = get_potential_features(np.copy(x), np.copy(y), classifier, np.copy(prev_masks), acquisition_cost)

    with tf.GradientTape() as tape:
        embedding_space, prediction, embedding = model(x*prev_masks)
        next_feature = tf.one_hot(next_feature, 21)
        loss = alpha * embedding_loss(distributions, embedding_space) + prediction_loss(next_feature, prediction)

    # Backprop
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

def validation_step(model, x, y, classifier, acquisition_cost, alpha=1):
    x, prev_masks = x_masked(x)

    _, distributions, next_feature = get_potential_features(
        x, y, classifier, np.copy(prev_masks), acquisition_cost
    )
    
    embedding_space, prediction, embedding = model(x * prev_masks)

    next_feature_onehot = tf.one_hot(next_feature, 21) 

    loss = (alpha * embedding_loss(distributions, embedding_space) 
            + prediction_loss(next_feature_onehot, prediction))
    return loss


X, X_val, y, y_val = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(10000).batch(512).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.batch(512).prefetch(tf.data.AUTOTUNE)

model = EmbeddingModels()
optimizer = tf.keras.optimizers.Adam(1e-3)

acquisition_cost = 0.015
num_epochs = 50
best_val_loss = float('inf')
for epoch in range(num_epochs):
    epoch_loss = 0.0
    step_count = 0

    for batch_x, batch_y in dataset:
        loss = train_step(model, optimizer, batch_x, batch_y, classifier, acquisition_cost)
        epoch_loss += loss.numpy()
        step_count += 1
    train_loss = epoch_loss / step_count

    val_loss_sum = 0.0
    val_steps = 0
    for val_x_batch, val_y_batch in val_dataset:
        loss_val = validation_step(model, val_x_batch, val_y_batch, classifier, acquisition_cost)
        val_loss_sum += loss_val.numpy()
        val_steps += 1
    val_loss = val_loss_sum / val_steps
    
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        model.save('/work/users/d/d/ddinh/aaco/models/embeddingembedding_synthetic_raw_gt.h5')
        model.save('/work/users/d/d/ddinh/aaco/models/embedding_synthetic_raw.keras')
        model.save_weights('/work/users/d/d/ddinh/aaco/models/embedding_synthetic_raw.weights.h5')

print("Training Complete.")
# add the regularizer same as divdis to avoid always predic as 0

/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Percentage of terminated:  0.518925518925519
Percentage of terminated:  0.5316455696202531
Percentage of terminated:  0.5486761710794298
Percentage of terminated:  0.554251012145749
Percentage of terminated:  0.5012224938875306
Percentage of terminated:  0.5449029126213593
Percentage of terminated:  0.5253061224489796
Percentage of terminated:  0.5739307535641548
Percentage of terminated:  0.5293637846655791
Percentage of terminated:  0.5304952926729432
Percentage of terminated:  0.5155993431855501
Percentage of terminated:  0.523673469387755
Percentage of terminated:  0.5317073170731708
Percentage of terminated:  0.5089942763695829
Percentage of terminated:  0.5192385581206966
Percentage of terminated:  0.5201301342008947
Percentage of terminated:  0.4975470155355683
Percentage of terminated:  0.5561866125760649
Percentage of terminated:  0.5097402597402597
Percentage of terminated:  0.5262301748678324
Percentage of terminated:  0.5534335635920358
Percentage of terminated:  0.48101265

2024-12-22 22:35:10.142347: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Percentage of terminated:  0.5093419983753046
Percentage of terminated:  0.5586682907023954
Percentage of terminated:  0.5257774140752864
Percentage of terminated:  0.5395918367346939
Percentage of terminated:  0.5314656922452294
Percentage of terminated:  0.5143782908059943
Percentage of terminated:  0.5091426249492076
Percentage of terminated:  0.495928338762215
Percentage of terminated:  0.5204918032786885
Percentage of terminated:  0.5334148329258354
Percentage of terminated:  0.5486365486365486
Percentage of terminated:  0.5177333876885446
Percentage of terminated:  0.5364859355890746
Percentage of terminated:  0.5255681818181818
Percentage of terminated:  0.49572649572649574
Percentage of terminated:  0.5048740861088545
Percentage of terminated:  0.532462229481421
Percentage of terminated:  0.5177849636216654
Percentage of terminated:  0.530296868645791


2024-12-22 22:45:38.817477: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

Percentage of terminated:  0.5238828967642527
Epoch 10, Train Loss: 2.7860, Validation Loss: 2.7436
Percentage of terminated:  0.5381184103811841
Percentage of terminated:  0.5422621478154349
Percentage of terminated:  0.5314285714285715
Percentage of terminated:  0.5004088307440719
Percentage of terminated:  0.524098825435399
Percentage of terminated:  0.530346232179226
Percentage of terminated:  0.521810028536486
Percentage of terminated:  0.5170731707317073
Percentage of terminated:  0.5552835577315381
Percentage of terminated:  0.5332517339861281
Percentage of terminated:  0.5353946297803092
Percentage of terminated:  0.5349025974025974
Percentage of terminated:  0.4967373572593801
Percentage of terminated:  0.5104551045510455
Percentage of terminated:  0.5540041067761807
Percentage of terminated:  0.5196914332115307
Percentage of terminated:  0.533469387755102
Percentage of terminated:  0.5519080837094789
Percentage of terminated:  0.5144485144485145
Percentage of terminated:  0.5

2024-12-23 11:33:27.684825: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Percentage of terminated:  0.4928774928774929
Percentage of terminated:  0.5432399512789281
Percentage of terminated:  0.509105625252934
Percentage of terminated:  0.5161290322580645
Percentage of terminated:  0.5002026753141467
Percentage of terminated:  0.513623424156161
Percentage of terminated:  0.5704863973619126
Percentage of terminated:  0.5224489795918368
Percentage of terminated:  0.5281632653061225
Percentage of terminated:  0.5543300653594772
Percentage of terminated:  0.5336872192731728
Percentage of terminated:  0.5820040899795501
Percentage of terminated:  0.5331695331695332
Percentage of terminated:  0.5102291325695582
Percentage of terminated:  0.5679414157851912
Percentage of terminated:  0.5245635403978888
Percentage of terminated:  0.4875864875864876
Percentage of terminated:  0.5616830065359477
Percentage of terminated:  0.5294117647058824
Percentage of terminated:  0.5523848348960456
Percentage of terminated:  0.5392798690671031
Percentage of terminated:  0.5319755

Percentage of terminated:  0.5408950617283951
Epoch 18, Train Loss: 2.7496, Validation Loss: 2.7145
Percentage of terminated:  0.5614320585842149
Percentage of terminated:  0.5154346060113729
Percentage of terminated:  0.5196238757154538
Percentage of terminated:  0.5319672131147541
Percentage of terminated:  0.5281314168377823
Percentage of terminated:  0.5278116118554608
Percentage of terminated:  0.5208503679476697
Percentage of terminated:  0.49756295694557273
Percentage of terminated:  0.5521899304134261
Percentage of terminated:  0.5062676910634857
Percentage of terminated:  0.5360323886639676
Percentage of terminated:  0.5254098360655738
Percentage of terminated:  0.5207993474714518
Percentage of terminated:  0.5150040551500406
Percentage of terminated:  0.5124948791478902
Percentage of terminated:  0.5181262729124236
Percentage of terminated:  0.5331423895253683
Percentage of terminated:  0.5170593013809911
Percentage of terminated:  0.5293159609120521
Percentage of terminated:

Percentage of terminated:  0.5860749808722264
Epoch 24, Train Loss: 2.7392, Validation Loss: 2.7097
Percentage of terminated:  0.5164567249085737
Percentage of terminated:  0.5214723926380368
Percentage of terminated:  0.5242758057935537
Percentage of terminated:  0.5244840145690004
Percentage of terminated:  0.5916700694160882
Percentage of terminated:  0.5268074735987003
Percentage of terminated:  0.5234981610134859
Percentage of terminated:  0.5391553915539156
Percentage of terminated:  0.4902120717781403
Percentage of terminated:  0.5166666666666667
Percentage of terminated:  0.5300613496932516
Percentage of terminated:  0.5406403940886699
Percentage of terminated:  0.5051041241322989
Percentage of terminated:  0.5002030044660982
Percentage of terminated:  0.5065359477124183
Percentage of terminated:  0.5390243902439025
Percentage of terminated:  0.5419222903885481
Percentage of terminated:  0.5326530612244897
Percentage of terminated:  0.5344195519348269
Percentage of terminated: 

In [6]:
model.save('/work/users/d/d/ddinh/aaco/models/embedding_gt.h5')
model.save('/work/users/d/d/ddinh/aaco/models/embedding_gt.keras')
model.save_weights('/work/users/d/d/ddinh/aaco/models/embedding_gt.weights.h5')

In [ ]:
NUM_MASKS_CURRENT = 100
NUM_MASKS_FUTURE = 100
NUM_TS = 12
NUM_MODALITIES = 4
acquisition_cost = 0.1
d = NUM_TS * NUM_MODALITIES
loss_function = nn.CrossEntropyLoss(reduction='none')

print("computing masks...")

ts_current = np.random.randint(0, num_ts, NUM_MASKS_CURRENT)
b_c = []
for i in range(NUM_MASKS_CURRENT):
    mask_temp = []
    for j in range(NUM_MODALITIES): 
        mask = np.sum(np.random.permutation(np.eye(NUM_TS))[:,:ts_current[i]+1][:, :np.random.randint(ts_current[i]+1)], 1)
        mask_temp.append(mask)
    b_c.append(mask_temp)
b_c = np.array(b_c)
b_c = np.transpose(b_c, (0, 2, 1)).reshape(-1, d)
b_empty = np.sum(b_c, 1) == 0
b_c = b_c[~b_empty]
b_c = np.unique(b_c, axis=0)
num_mask = b_c.shape[0]


x_current = np.repeat(train_x, num_mask, axis=0)
y_current = np.repeat(train_y, num_mask, axis=0)

b_current = np.concatenate([b_c for _ in range(train_x.shape[0])], axis=0)
print("computing predictions...")

y_pred = np.zeros(y_current.shape)
for ts in range(NUM_TS):
    print(ts)
    x_input = np.zeros(x_current.shape)
    mask_input = np.zeros(b_current.shape)
    
    for k in range(NUM_MODALITIES):
        x_input[:,k * NUM_TS: k * NUM_TS + ts + 1] = np.copy(x_current[:,k * NUM_TS: k * NUM_TS + ts + 1])
        mask_input[:,k * NUM_TS: k * NUM_TS + ts + 1] = np.copy(b_current[:,k * NUM_TS: k * NUM_TS + ts + 1])
    
    x_input = torch.Tensor(x_input)
    mask_input = torch.Tensor(mask_input.numpy())
    ts_rep = np.repeat(ts, x_input.shape[0]).reshape(-1, 1)
    pred = classifier.predict(np.concatenate([x_input * mask_input, ts_rep], axis=-1), verbose=0)
    y_pred[:,ts,:] = pred
y_pred = torch.Tensor(y_pred)
y_current = torch.Tensor(y_current)
print("computing loss...")
cost = torch.zeros(b_current.shape[0])
for m in range(NUM_MODALITIES):
    modality_cost = 1 if m in [0, 1] else 0.5
    cost += b_current[:, m * NUM_TS: (m + 1) * NUM_TS].sum(1) * acquisition_cost * modality_cost
loss_current = compute_accumulated_loss(y_pred, y_current, loss_function) + cost 

computing masks...
computing predictions...
0
1
2
3
4
5
6
7


KeyboardInterrupt: 

In [ ]:
print("computing best mask...")
# while loop here to make sure mask is not 0
Bf = []
for i in range(NUM_MASKS_CURRENT):
    for j in range(NUM_MASKS_FUTURE): 
        mask_temp = []
        for j in range(NUM_MODALITIES): 
            mask = np.sum(np.random.permutation(np.eye(NUM_TS))[:,ts_current[i]+1:][:, :np.random.randint(NUM_TS-ts_current[i]+1)], 1)
            mask_temp.append(mask)
        Bf.append(mask_temp)
Bf = np.array(Bf)
Bf = np.transpose(Bf, (0, 2, 1)).reshape(-1, d)
    
Bf[0] = np.zeros(d) # current mask 
Bf = np.unique(Bf, axis=0)

num_mask_future = Bf.shape[0]
 
x_future = np.repeat(x_current, num_mask_future, axis=0)
y_future = np.repeat(y_current, num_mask_future, axis=0)
print("x_future shape: ", x_future.shape)

b_current_for_future = np.repeat(b_c, num_mask_future, axis=0)


computing best mask...
x_future shape:  (6355800, 48)


In [8]:
Bforb = np.repeat(Bf, num_mask, axis=0)

Bforb = np.minimum(b_current_for_future+Bforb, 1)
Bforb = np.concatenate([Bforb for _ in range(train_x.shape[0])], axis=0)

In [13]:
print("computing predictions...")
y_pred = np.zeros(y_future.shape)
for ts in range(NUM_TS):
    print(ts)
    x_input = np.zeros(x_future.shape)
    mask_input = np.zeros(Bforb.shape)
    
    for k in range(NUM_MODALITIES):
        x_input[:,k * NUM_TS: k * NUM_TS + ts + 1] = np.copy(x_future[:,k * NUM_TS: k * NUM_TS + ts + 1])
        mask_input[:,k * NUM_TS: k * NUM_TS + ts + 1] = np.copy(Bforb[:,k * NUM_TS: k * NUM_TS + ts + 1])
    
    x_input = torch.Tensor(x_input)
    mask_input = torch.Tensor(mask_input)
    ts_rep = np.repeat(ts, x_input.shape[0]).reshape(-1, 1)
    pred = classifier.predict(np.concatenate([x_input * mask_input, ts_rep], axis=-1), verbose=1)
    y_pred[:,ts,:] = pred

print("computing loss...")    
y_pred = torch.Tensor(y_pred)
y_future = torch.Tensor(y_future)
cost = torch.zeros(Bforb.shape[0])
for m in range(NUM_MODALITIES):
    modality_cost = 1 if m in [0, 1] else 0.5
    cost += Bforb[:, m * NUM_TS: (m + 1) * NUM_TS].sum(1) * acquisition_cost * modality_cost
loss_future = compute_accumulated_loss(y_pred, y_future, loss_function) + cost 


computing predictions...
0
198619/198619 ━━━━━━━━━━━━━━━━━━━━ 78s 394us/step
1
198619/198619 ━━━━━━━━━━━━━━━━━━━━ 77s 389us/step
2
198619/198619 ━━━━━━━━━━━━━━━━━━━━ 78s 394us/step
3
 62848/198619 ━━━━━━━━━━━━━━━━━━━━ 51s 383us/step

KeyboardInterrupt: 

In [15]:
masksper = 256 
d = train_x_ts.shape[1] - 1
X_class = np.concatenate([train_x_ts]*masksper, 0)
Y_class = np.concatenate([train_y_ts]*masksper, 0)
B = np.concatenate(
[np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(X_class.shape[0])],
1)
B = np.float32(B.T)

zero_mask = X_class[:,:d] == 0
B[zero_mask] = 0
# remove 0 mask
mask_nonzero = np.sum(B, axis=1) != 0
B = B[mask_nonzero]
X_class = X_class[mask_nonzero]
Y_class = Y_class[mask_nonzero]

X_class[:,:d] = X_class[:,:d] * B
X_class.shape, Y_class.shape, B.shape

[758316. 758987. 758008. 757953. 757620. 757955. 757972. 758435. 757102.
 756993. 757385. 757279. 757917. 757722. 757728. 757501. 757812. 758526.
 758420. 757669. 758158. 758848. 758452. 758176. 757722. 758034. 758743.
 759001. 758049. 757567. 758648. 758536. 757489. 756505. 757903. 757405.
 757466. 757965. 758633. 757916. 758311. 757459. 758096. 757114. 757111.
 757801. 758337. 757861.]


((1418396, 49), (1418396, 3), (1418396, 48))

In [ ]:
ts_current = np.random.randint(0, num_ts, NUM_MASKS_CURRENT)
b_current = []
for i in range(NUM_MASKS_CURRENT):
    mask_temp = []
    for j in range(NUM_MODALITIES): 
        mask = np.sum(np.random.permutation(np.eye(NUM_TS))[:,:ts_current[i]+1][:, :np.random.randint(ts_current[i]+1)], 1)
        mask_temp.append(mask)
    b_current.append(mask_temp)
    
b_current = np.array(b_current)
b_current = np.transpose(b_current, (1, 0, 2)).reshape(-1, NUM_MODALITIES * NUM_TS)
b_empty = np.sum(b_current, 1) == 0
b_current = b_current[~b_empty]

In [7]:
masksper = 5
d = val_x_ts.shape[1]
X_class_val = np.concatenate([val_x_ts]*masksper, 0)
Y_class_val = np.concatenate([val_y_ts]*masksper, 0)
B_val = np.concatenate(
[np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(X_class_val.shape[0])],
1)
B_val = np.float32(B_val.T)

# B_val = np.ones_like(B_val)

zero_mask_val = X_class_val[:,:d] == 0
B_val[zero_mask_val] = 0

# remove 0 mask
mask_nonzero_val = np.sum(B_val, axis=1) != 0
B_val = B_val[mask_nonzero_val]
X_class_val = X_class_val[mask_nonzero_val]
Y_class_val = Y_class_val[mask_nonzero_val]

X_class_val[:,:d] = X_class_val[:,:d] * B_val
X_class_val.shape, Y_class_val.shape, B_val.shape

((5595, 49), (5595, 3), (5595, 49))

In [10]:
Y_class.shape

(1132588, 3)

In [ ]:
y = np.argmax(logits, axis=1).astype(np.int32)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
from tensorflow.keras.metrics import AUC

TOTAL_FEATURES = train_x_ts.shape[-1]
HIDDEN_DIM = 10
NUM_CLASSES = train_y_ts.shape[-1]
DROPOUT_RATE = 0.75

def create_mlp():
    model = models.Sequential([
        layers.Input(shape=(TOTAL_FEATURES,)),
        layers.Dense(HIDDEN_DIM, activation='relu'),
        # layers.Dropout(DROPOUT_RATE),
        layers.Dense(HIDDEN_DIM, activation='relu'),
        # layers.Dropout(DROPOUT_RATE),
        layers.Dense(NUM_CLASSES, activation='softmax')  
    ])
    return model



# Compile the model
model = create_mlp()
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy', AUC(name='roc_auc'), AUC(name='prc', curve='PR')]
)

def train_model(model, X_train, y_train, X_val, y_val, epochs=100, batch_size=32):
    checkpoint = ModelCheckpoint(
        '/work/users/d/d/ddinh/aaco/models/mlp_embedding.keras',  
        monitor='val_roc_auc',
        mode='max',  
        save_best_only=True,
        verbose=1
    )
    early_stopping = EarlyStopping(
        monitor='val_roc_auc', 
        mode='max', 
        patience=10,  
        verbose=1
    )
    with tf.device('/device:GPU:0'):
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[checkpoint, early_stopping],
            verbose=1
        )
    return history

def evaluate_model(model, X, y):
    loss, accuracy, roc_auc, prc = model.evaluate(X, y, verbose=0)
    print(f"Loss: {loss}, Accuracy: {accuracy}, ROC AUC: {roc_auc}, PRC: {prc}")
    return loss, accuracy, roc_auc

# history = train_model(model, train_x_ts, train_y_ts, val_x_ts, val_y_ts)
# evaluate_model(model, val_x_ts, val_y_ts)
history = train_model(model, X_class, Y_class, X_class_val, Y_class_val)
evaluate_model(model, X_class_val, Y_class_val)

def plot_history(history):
    plt.plot(history.history['roc_auc'])
    plt.plot(history.history['val_roc_auc'])
    plt.title('model roc_auc')
    plt.ylabel('roc_auc')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['prc'])
    plt.plot(history.history['val_prc'])
    plt.title('model prc')
    plt.ylabel('prc')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
plot_history(history)
#  val_prc: 0.5998 - val_roc_auc: 0.7917

Epoch 1/100
35373/35377 ━━━━━━━━━━━━━━━━━━━━ 0s 915us/step - accuracy: 0.6022 - loss: 0.7826 - prc: 0.6291 - roc_auc: 0.8070
Epoch 1: val_roc_auc improved from -inf to 0.76294, saving model to /work/users/d/d/ddinh/aaco/models/mlp_test.keras
35377/35377 ━━━━━━━━━━━━━━━━━━━━ 42s 1ms/step - accuracy: 0.6022 - loss: 0.7826 - prc: 0.6291 - roc_auc: 0.8070 - val_accuracy: 0.5645 - val_loss: 0.9315 - val_prc: 0.5431 - val_roc_auc: 0.7629
Epoch 2/100
35363/35377 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6201 - loss: 0.7467 - prc: 0.6643 - roc_auc: 0.8241
Epoch 2: val_roc_auc improved from 0.76294 to 0.76604, saving model to /work/users/d/d/ddinh/aaco/models/mlp_test.keras
35377/35377 ━━━━━━━━━━━━━━━━━━━━ 45s 1ms/step - accuracy: 0.6201 - loss: 0.7467 - prc: 0.6643 - roc_auc: 0.8241 - val_accuracy: 0.5657 - val_loss: 0.9197 - val_prc: 0.5514 - val_roc_auc: 0.7660
Epoch 3/100
35340/35377 ━━━━━━━━━━━━━━━━━━━━ 0s 936us/step - accuracy: 0.6223 - loss: 0.7426 - prc: 0.6686 - roc_auc: 0.8261
Ep

KeyboardInterrupt: 

My version

In [ ]:

dataset = load_adni_data()
x = dataset.x
y = dataset.y

mask_nan = np.isnan(x)

x[mask_nan] = 0

num_ts = y.shape[1]


In [ ]:
for i in range(num_ts):
    count = [0,0,0]
    total = 0
    for j in range(y.shape[0]):
        for k in range(3):
            if y[j,i,k] == 1:
                count[k] += 1
                total += 1
    count = np.array(count)
    print(count/total)

[0.06387226 0.64271457 0.29341317]
[0.08805668 0.61234818 0.29959514]
[0.10761421 0.59187817 0.30050761]
[0.125      0.58974359 0.28525641]
[0.16613757 0.54179894 0.29206349]
[0.13573883 0.56872852 0.29553265]
[0.178125  0.5796875 0.2421875]
[0.18283582 0.53731343 0.27985075]
[0.17350746 0.46455224 0.3619403 ]
[0.18235294 0.45294118 0.36470588]
[0.18596491 0.49473684 0.31929825]
[0.17098446 0.43005181 0.39896373]


In [14]:
count = [0,0,0]
total = 0
for i in range(num_ts):
    for j in range(y.shape[0]):
        for k in range(3):
            if y[j,i,k] == 1:
                count[k] += 1
                total += 1
count = np.array(count)
print(count/total)

[0.13054449 0.56998672 0.29946879]


In [4]:
mask_nan = x==0
# get the number of nan values
total = mask_nan.sum()
probability = total / x.size
probability

0.40976380572188953

In [5]:
# split dataset into train and test sets 80/20 with random seed 42
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((801, 12, 4), (201, 12, 4), (801, 12, 3), (201, 12, 3))

In [5]:
# x_train = prepare_time_series(torch.Tensor([range(0,12)] * x_train.shape[0]), torch.Tensor(x_train))[1]
# y_train = prepare_time_series(torch.Tensor([range(0,12)] * y_train.shape[0]), torch.Tensor(y_train))[1]

In [6]:
# x_train = torch.cat([x_train, x_train[:, [-1]]], dim=-2).numpy()
# y_train = torch.cat([y_train, y_train[:, [-1]]], dim=-2).numpy()

In [7]:
# y_train = np.round(y_train)

In [8]:
# x_ts = []
# y_ts = []

# for i in range(num_ts): 
#     x_temp = np.zeros_like(x_train)
#     x_temp[:, :i+1, :] = x_train[:, :i+1, :]
#     x_ts.append(x_temp)    
#     y_ts.append(y_train)
# x_ts = np.concatenate(x_ts, axis=0)
# y_ts = np.concatenate(y_ts, axis=0)
# x_ts.shape, y_ts.shape

In [6]:
"""temp
"""
x_ts = x_train
y_ts = y_train

In [7]:
x_all = []
for item in x_ts:
    x_all.append(item.flatten('F'))
x_train = np.array(x_all)
y_train = y_ts

In [ ]:
import numpy as np

def apply_masks(inputs, pm=0.2, pd_m=0.4):
    inputs = np.copy(inputs).reshape(-1, 12, 4)
    n, T, M = inputs.shape
    masked_inputs = inputs.copy()

    mask_m1 = np.random.binomial(1, pm, size=(n, T, M))
    
    t_max = np.random.randint(0, T, size=n)
    mask_m2 = np.ones((n, T, M))
    for i in range(n):
        mask_m2[i, t_max[i]:] = 0
    
    modality_drop = np.random.binomial(1, pd_m, size=(n, M))
    mask_m3 = np.ones((n, T, M))
    for m in range(M):
        mask_m3[:, :, m] *= modality_drop[:, m].reshape(-1, 1)

    final_mask = mask_m1 * mask_m2 * mask_m3
    masked_inputs *= final_mask

    return masked_inputs, final_mask.reshape(n, -1)

pm = 0.2
pd_m = 0.4 

In [ ]:
masksper = 256 * 2
d = x_train.shape[1]
X_class = np.concatenate([x_train]*masksper, 0)
Y_class = np.concatenate([y_train]*masksper, 0)
# B = np.concatenate(
# [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(X_class.shape[0])],
# 1)
# B = np.float32(B.T)

_, B = apply_masks(X_class, pm, pd_m)
    
"""
remove for interpolation
"""
zero_mask = X_class == 0
B[zero_mask] = 0

# # remove 0 mask
mask_nonzero = np.sum(B, axis=1) != 0
B = B[mask_nonzero]
X_class = X_class[mask_nonzero]
Y_class = Y_class[mask_nonzero]


In [12]:
# mask_nan = X_class == 0
# B[mask_nan] = 0

In [16]:
X_class = np.concatenate((X_class*B, B), 1)

In [17]:
X_class.shape, Y_class.shape

((213901, 96), (213901, 12, 3))

In [18]:
location = []
for i in range(B.shape[0]):
    b = B[i]
    temp = -1
    for j in range(4):
        part = b[num_ts*j:num_ts*(j+1)]
        max_index = np.where(part == 1)[0]
        if len(max_index) > 0:
            max_index = max_index[-1]
            if max_index > temp:
                temp = max_index
    location.append(temp)
    
location = np.array(location)

In [19]:
unique, counts = np.unique(location, return_counts=True)
print(np.asarray((unique, counts)).T)

[[    0 16433]
 [    1 22512]
 [    2 24842]
 [    3 30042]
 [    4 27878]
 [    5 18594]
 [    6 21267]
 [    7  9046]
 [    8 19186]
 [    9  5576]
 [   10 10502]
 [   11  8023]]


In [20]:
y_temp = []
for i in range(Y_class.shape[0]):
    y_location = Y_class[i, location[i]]
    mask_nan = np.isnan(y_location)
    y_location[mask_nan] = 0
    y_temp.append(y_location)
y_temp = np.array(y_temp)

In [21]:
# remove y with nan values
mask_zero = np.sum(y_temp, axis=1) != 0
X_class = X_class[mask_zero]
Y_class = y_temp[mask_zero]

In [22]:
Y_class = np.argmax(Y_class, axis=1)

In [23]:
# Train classifier
est = XGBClassifier(n_estimators=256, device='gpu')
est.fit(X_class, Y_class)

/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [10:37:46] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [10:37:46] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='gpu', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=256, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
# est.save_model('/work/users/d/d/ddinh/aaco/models/adni_different_masking.model')

/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [10:39:02] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [25]:
x_ts_val = []
y_ts_val = []

for i in range(num_ts): 
    x_temp = np.zeros_like(x_val)
    x_temp[:, :i+1, :] = x_val[:, :i+1, :]
    x_ts_val.append(x_temp)    
    y_ts_val.append(y_val)
x_ts_val = np.concatenate(x_ts_val, axis=0)
y_ts_val = np.concatenate(y_ts_val, axis=0)
x_ts_val.shape, y_ts_val.shape

((2412, 12, 4), (2412, 12, 3))

In [26]:
x_all_val = []
for item in x_ts_val:
    x_all_val.append(item.flatten('F'))
x_val = np.array(x_all_val)
y_val = y_ts_val

In [27]:
masksper = 1
d = x_val.shape[1]
X_class_val = np.concatenate([x_val]*masksper, 0)
Y_class_val = np.concatenate([y_val]*masksper, 0)
B_val = np.concatenate(
[np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(X_class_val.shape[0])],
1)
B_val = np.float32(B_val.T)

# B_val = np.ones_like(B_val)

zero_mask_val = X_class_val == 0
B_val[zero_mask_val] = 0

# remove 0 mask
mask_nonzero_val = np.sum(B_val, axis=1) != 0
B_val = B_val[mask_nonzero_val]
X_class_val = X_class_val[mask_nonzero_val]
Y_class_val = Y_class_val[mask_nonzero_val]

In [28]:
location_val = []
for i in range(B_val.shape[0]):
    b = B_val[i]
    temp = -1
    for j in range(4):
        part = b[num_ts*j:num_ts*(j+1)]
        max_index = np.where(part == 1)[0]
        if len(max_index) > 0:
            max_index = max_index[-1]
            if max_index > temp:
                temp = max_index
    location_val.append(temp)
    
location_val = np.array(location_val)

In [29]:
y_temp_val = []
for i in range(Y_class_val.shape[0]):
    y_location_val = Y_class_val[i, location_val[i]]
    mask_nan_val = np.isnan(y_location_val)
    y_location_val[mask_nan_val] = 0
    y_temp_val.append(y_location_val)
y_temp_val = np.array(y_temp_val)

In [30]:
# remove y with nan values for validation set
mask_zero_val = np.sum(y_temp_val, axis=1) != 0
X_class_val = X_class_val[mask_zero_val]
Y_class_val = y_temp_val[mask_zero_val]
B_val = B_val[mask_zero_val]

In [31]:
mask_zero_val.shape

(2246,)

In [32]:
X_class_val = np.concatenate((X_class_val*B_val, B_val), 1)

In [33]:
val_preds = est.predict_proba(X_class_val) 

In [34]:
print(np.mean(np.argmax(val_preds, 1)==np.argmax(Y_class_val, 1)))

0.5409617097061442


In [35]:
# this is temoprary evaluation, since it's not really correct.
# it should be evaluated on the accumulated predictions, not the individual predictions like this rolling out 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

pr_auc_macro = average_precision_score(Y_class_val, val_preds, average="macro")
print(pr_auc_macro)

roc_auc_macro = roc_auc_score(Y_class_val, val_preds, average="macro", multi_class="ovr")
print(roc_auc_macro)

0.5109105973299515
0.7030657167468642


In [36]:
np.sum(B_val.mean(0))

10.037846

In [ ]:
# baseline
# 0.5057228523811975
# 0.6798018971723648

# masking from a2mt paper
# 0.5109105973299515
# 0.7030657167468642